In [1]:
from src.constants import *
from src.model.my_model import MyModel
from src.model.baselines import UNet, SWINUNETR
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity
from src.model.blocks import *
import torch.nn as nn
from src.utils.metrics import *

In [13]:
example2 = torch.rand(size=[4, 1, 128, 128, 128]).cuda()
mask = torch.rand(size=[4, 3, 128, 128, 128]).cuda()
loss_fn = monai.losses.DiceCELoss()

In [3]:
# modes = normal, skip, autoencoder, half_half, no_vit
model = MyModel(in_channels=1,
                 out_channels=3,
                 lower_channels=32,
                 big_channel=32,
                 patch_size=8,
                 embed_dim=256,
                 mode="normal").cuda()
optimizer = torch.optim.Adam(params=model.parameters(), lr=3e-4)

# model = UNet.cuda()

In [14]:
%%time
out = model(example2)
l = loss_fn(out, mask)
l.backward()
optimizer.step()

CPU times: user 759 ms, sys: 343 ms, total: 1.1 s
Wall time: 1.15 s


In [5]:
import os

In [6]:
os.system("nvidia-smi")

Mon May  8 13:33:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.86.01    Driver Version: 515.86.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A40          On   | 00000000:3B:00.0 Off |                    0 |
|  0%   43C    P8    34W / 300W |      0MiB / 46068MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A40          On   | 00000000:5E:00.0 Off |                    0 |
|  0%   

0

In [7]:
out.shape

torch.Size([4, 3, 128, 128, 128])

In [8]:
count_learnable_parameters(model)

55688990

In [12]:
count_learnable_parameters(model.vit)

54951722

In [10]:
count_learnable_parameters(SWINUNETR)

15703029

In [11]:
count_learnable_parameters(UNet)

7164975